In [2]:
import os
import tensorflow as tf

import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

In [7]:
vgg = tf.keras.applications.VGG16(include_top=False, input_shape=(400, 400, 3), weights='imagenet')
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 400, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 400, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 200, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 200, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 200, 200, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 100, 100, 128)     0     

In [8]:
vgg.input

<KerasTensor: shape=(None, 400, 400, 3) dtype=float32 (created by layer 'input_3')>

In [9]:
layer = vgg.get_layer('block5_conv1')
print(layer)

In [11]:
output = vgg.get_layer('block5_conv1').output
print(output.shape)

(None, 25, 25, 512)


In [12]:
output = vgg.get_layer('block5_conv2').output
print(output.shape)

(None, 25, 25, 512)


In [13]:
output = vgg.get_layer('block5_conv3').output
print(output.shape)

(None, 25, 25, 512)


In [15]:
outputs = [vgg.get_layer('block5_conv1').output, vgg.get_layer('block5_conv2').output]
print(outputs)

[<KerasTensor: shape=(None, 25, 25, 512) dtype=float32 (created by layer 'block5_conv1')>, <KerasTensor: shape=(None, 25, 25, 512) dtype=float32 (created by layer 'block5_conv2')>]


In [16]:
content_layers = ['block5_conv1']

In [17]:
def get_vgg_layers(layer_names):
    vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet')
    vgg.trainable = False

    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], [outputs])

    return model

In [18]:
# create the model:
content_layers = ['block5_conv2']

content_extractor = get_vgg_layers(content_layers)
content_extractor.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [19]:
data = tf.random.normal((1, 400, 400, 3))
content_outputs = content_extractor(data)

print(type(content_outputs))
print(len(content_outputs))
print(content_outputs[0].shape)

<class 'list'>
1
(1, 25, 25, 512)


In [20]:

# Look at the statistics of each layer's output
for name, output in zip(content_layers, content_outputs):
    print(name)
    print("  shape: ", output.numpy().shape)
    print("  min: ", output.numpy().min())
    print("  max: ", output.numpy().max())
    print("  mean: ", output.numpy().mean())
    print()

block5_conv2
  shape:  (1, 25, 25, 512)
  min:  0.0
  max:  4.7887263
  mean:  0.13485871

